In [61]:
import pandas as pd
import numpy as np

In [62]:
# Load CSV
file_path = "data/hdfc-bank-raw.csv"
df = pd.read_csv(file_path)
df.tail()

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades
245,28-Jan-2025,EQ,"1,647.00","1,684.30","1,647.00","1,629.80","1,673.30","1,670.40","1,670.57","1,880.00","1,363.55","1,50,89,656","25,20,82,71,808.85","3,50,835"
246,29-Jan-2025,EQ,"1,675.00","1,681.15","1,668.85","1,670.40","1,676.00","1,677.30","1,675.45","1,880.00","1,363.55","63,09,945","10,57,19,94,646.45","1,96,187"
247,30-Jan-2025,EQ,"1,681.00","1,697.90","1,672.55","1,677.30","1,693.10","1,691.50","1,685.73","1,880.00","1,363.55","1,28,72,423","21,69,94,04,205.05","2,63,161"
248,31-Jan-2025,EQ,"1,687.25","1,704.70","1,683.95","1,691.50","1,699.50","1,698.75","1,696.52","1,880.00","1,363.55","79,10,843","13,42,08,73,179.40","2,15,938"
249,01-Feb-2025,EQ,"1,697.50","1,713.00","1,676.05","1,698.75","1,692.00","1,690.95","1,694.22","1,880.00","1,363.55","27,93,764","4,73,32,38,897.95","88,839"


In [63]:
df.columns = df.columns.str.strip()  # Removes spaces

In [64]:
##### ADD DAY LOGIC HERE FOR WEEK SEPARATION #####
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')

# # Add 'Day' column
df['Day'] = df['Date'].dt.day_name()

# Sort by Date (ascending order for correct weekly grouping)
df = df.sort_values(by='Date')

In [65]:
df.head()

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades,Day
0,2024-02-02,EQ,"1,475.00","1,480.85","1,442.05","1,466.35","1,445.90","1,446.15","1,461.81","1,757.50","1,380.25","2,24,33,877","32,79,41,41,061.40","5,47,098",Friday
1,2024-02-05,EQ,"1,446.00","1,451.90","1,434.05","1,446.15","1,443.05","1,444.85","1,442.08","1,757.50","1,380.25","1,93,02,523","27,83,57,69,812.35","4,87,034",Monday
2,2024-02-06,EQ,"1,445.55","1,449.70","1,432.60","1,444.85","1,443.55","1,444.10","1,442.11","1,757.50","1,380.25","2,05,37,870","29,61,79,40,997.40","4,10,774",Tuesday
3,2024-02-07,EQ,"1,453.00","1,453.80","1,426.65","1,444.10","1,430.00","1,429.95","1,435.86","1,757.50","1,380.25","2,74,18,543","39,36,90,92,511.05","5,23,984",Wednesday
4,2024-02-08,EQ,"1,429.85","1,438.95","1,400.50","1,429.95","1,407.25","1,403.05","1,415.51","1,757.50","1,380.25","3,45,53,062","48,91,02,44,009.15","7,97,559",Thursday


In [66]:
# Function to insert empty rows
def insert_empty_rows(df):
    empty_row = pd.DataFrame([[""] * len(df.columns)], columns=df.columns)
    df_with_gaps = pd.concat([pd.concat([df.iloc[[i]], empty_row], ignore_index=True) for i in range(len(df))], ignore_index=True)
    return df_with_gaps

In [67]:
# Group by weeks (Monday-Sunday)
# df['Week'] = df['Date'].dt.to_period('W').astype(str)
df['Week'] = df['Date'].dt.isocalendar().week  # Get ISO week number (Monday-Sunday)

grouped = [insert_empty_rows(g) for _, g in df.groupby('Week')]

In [68]:
# Combine the weekly separated data
df.head()

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades,Day,Week
0,2024-02-02,EQ,"1,475.00","1,480.85","1,442.05","1,466.35","1,445.90","1,446.15","1,461.81","1,757.50","1,380.25","2,24,33,877","32,79,41,41,061.40","5,47,098",Friday,5
1,2024-02-05,EQ,"1,446.00","1,451.90","1,434.05","1,446.15","1,443.05","1,444.85","1,442.08","1,757.50","1,380.25","1,93,02,523","27,83,57,69,812.35","4,87,034",Monday,6
2,2024-02-06,EQ,"1,445.55","1,449.70","1,432.60","1,444.85","1,443.55","1,444.10","1,442.11","1,757.50","1,380.25","2,05,37,870","29,61,79,40,997.40","4,10,774",Tuesday,6
3,2024-02-07,EQ,"1,453.00","1,453.80","1,426.65","1,444.10","1,430.00","1,429.95","1,435.86","1,757.50","1,380.25","2,74,18,543","39,36,90,92,511.05","5,23,984",Wednesday,6
4,2024-02-08,EQ,"1,429.85","1,438.95","1,400.50","1,429.95","1,407.25","1,403.05","1,415.51","1,757.50","1,380.25","3,45,53,062","48,91,02,44,009.15","7,97,559",Thursday,6


In [69]:
# Save the processed CSV
output_path = "output/step-1-weeks-added.csv"
df.to_csv(output_path, index=False)

print(f"CSV processed and saved as {output_path}.")

CSV processed and saved as output/step-1-weeks-added.csv.
